In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
lg = pd.read_json('data_legitimate_36400.json')
lg.head(10)

,0
0,http://www.conceptdraw.com/How-To-Guide/Local-...
1,https://www.edrawsoft.com/Local-Area-Network.php
2,http://www.webopedia.com/TERM/L/local_area_net...
3,https://www.acsac.org/secshelf/book001/16.pdf
4,http://www.diffen.com/difference/LAN_vs_WAN
5,http://sentence.yourdictionary.com/lan
6,http://www.itrelease.com/2012/12/examples-and-...
7,http://www.dictionary.com/browse/lan
8,http://fcit.usf.edu/Network/chap1/chap1.htm
9,http://www.Cisco.com/c/en/us/support/wireless/...


In [3]:
ph = pd.read_json('data_phishing_37175.json')
ph.head(10)

,0
0,apple-iforget.com
1,safe-id-login.com
2,manage.netflix.com.usermanagement.key.1973573....
3,manage.netflix.com.usermanagement.key.1973574....
4,helpplusinfo01.com
5,icloudsupporto.com
6,fbverify.biz
7,paypal-my-cash.com
8,scotiabank-security.com
9,user-appleid-find.com


### Changing column name

In [4]:
lg.rename(columns = {0 : 'URLs'}, inplace = True)
print (len(lg))
lg.head(10)

36400


,URLs
0,http://www.conceptdraw.com/How-To-Guide/Local-...
1,https://www.edrawsoft.com/Local-Area-Network.php
2,http://www.webopedia.com/TERM/L/local_area_net...
3,https://www.acsac.org/secshelf/book001/16.pdf
4,http://www.diffen.com/difference/LAN_vs_WAN
5,http://sentence.yourdictionary.com/lan
6,http://www.itrelease.com/2012/12/examples-and-...
7,http://www.dictionary.com/browse/lan
8,http://fcit.usf.edu/Network/chap1/chap1.htm
9,http://www.Cisco.com/c/en/us/support/wireless/...


In [5]:
ph.rename(columns = {0 : 'URLs'}, inplace = True)
ph.head(10)

,URLs
0,apple-iforget.com
1,safe-id-login.com
2,manage.netflix.com.usermanagement.key.1973573....
3,manage.netflix.com.usermanagement.key.1973574....
4,helpplusinfo01.com
5,icloudsupporto.com
6,fbverify.biz
7,paypal-my-cash.com
8,scotiabank-security.com
9,user-appleid-find.com


### Adding a column 'isPhishing' to store the Phishing Result

#### isPhishing = 0 for legitimate websites
#### isPhishing = 1 for phishing websites


In [6]:
lg["isPhishing"] = 0
lg.sort_values('URLs', inplace = True)
lg.drop_duplicates(inplace = True, subset = 'URLs')
lg.head(10)


,URLs,isPhishing
21891,http://0cu203.bankruptcy-law-help.com/15593752...,0
21795,http://1-drivers.org/videocards/diamond_multim...,0
5135,http://1000islandswritersfestival.ca/wp-conten...,0
603,http://1000projects.org/fundamentals-of-passiv...,0
7998,http://1000projects.org/technical-seminar-topi...,0
10734,http://103.ip-144-217-81.net/documents/graphic...,0
36378,http://103.ip-144-217-81.net/documents/search-...,0
27969,http://103.ip-144-217-81.net/documents/the-hel...,0
17670,http://10beasts.com/best-3d-pens/,0
11321,http://10beasts.com/best-wireless-routers/,0


In [7]:
ph["isPhishing"] = 1
ph.sort_values('URLs', inplace = True)

ph.drop_duplicates(inplace = True, subset = 'URLs')
ph.head(10)

,URLs,isPhishing
111,13bmosecure.com,1
155,34zy2.sljtm.com,1
108,accountverification.online,1
50,apaypal.co/Secure/PP/Paypal/,1
33,app-findmyiphone.com,1
142,apple-accounts.me/clients/home/,1
39,apple-icloudcid.com,1
54,apple-id-support.info,1
20359,apple-iforget.com,1
166,apple-inc-help.com,1


### Merging the 2 dataframes into a single dataframe for pre-processing

In [8]:
df = pd.concat([ph, lg])


print (len(df))
df.head(10)

54763


,URLs,isPhishing
111,13bmosecure.com,1
155,34zy2.sljtm.com,1
108,accountverification.online,1
50,apaypal.co/Secure/PP/Paypal/,1
33,app-findmyiphone.com,1
142,apple-accounts.me/clients/home/,1
39,apple-icloudcid.com,1
54,apple-id-support.info,1
20359,apple-iforget.com,1
166,apple-inc-help.com,1


## Feature Selection

### Calculating Raw Length of the urls

In [9]:
import re, statistics
def rawlength(s):
    return len(s)

def isHTTPS(s):
    if(s.startswith('https')):
        return 1
    else:
        return 0

def hasWWW(s):
    if 'www' in s:
        return 1
    else:
        return 0

def hasCOM(s):
    if '.com' in s:
        return 1
    else:
        return 0

def lenMaxWord(s):
    words = re.split('[. -/_]',s)
    m = 0
    for i in words:
        m = max(m, len(i))
    return m


def lenAvgWord(s):
    words = re.split('[. -/_]',s)
    l = []
    for i in words:
        if (len(i)>0):
            l.append(len(i))
    return round(sum(l)/len(l))

def lenMinWord(s):
    words = re.split('[. -/_]',s)
    l = []
    for i in words:
        if (len(i)>0):
            l.append(len(i))
    return min(l)

def SDlengthWord(s):
    words = re.split('[. -/_]',s)
    l = []
    for i in words:
        if (len(i)>0):
            l.append(len(i))
    return statistics.stdev(l)

def numOfSubdom(site):
    s = re.search("//",site)
    if s:
        site = site[s.end():]
    e = re.search("/",site)
    if e:
        site = site[:e.end()-1]
    site = site.split(".")
    # one for domain extension eg. com,edu,in 
    num = len(site)-1
    if 'www' in site:
        num = num-1
    return num

def keyCount(site):
    count = 0
    wordList = re.split(r"\.|\/|-|\?|=|#|&|_|@|:\/\/",site)
    for word in wordList:
        if word in keywordList:
            count += 1
    return count

df["RawLength"] = df["URLs"].apply(rawlength)
df["HTTPS"] = df["URLs"].apply(isHTTPS)
df["WWW"] = df["URLs"].apply(hasWWW)
df[".COM"] = df["URLs"].apply(hasCOM)
df["largestWord"] = df["URLs"].apply(lenMaxWord)
df["avgLength"] = df["URLs"].apply(lenAvgWord)
df["minLength"] = df["URLs"].apply(lenMinWord)
df["SDlength"] = df["URLs"].apply(SDlengthWord)
df["countSubDomain"] = df["URLs"].apply(numOfSubdom)

df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgLength,minLength,SDlength,countSubDomain
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1


#### Checking if URL contains .com

#### Find maximum length of word in URL

#### Finding average length of word

#### Finding Minimum Length of a word in URL